# Notebook 4: Iterative EDA and Refinement

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from xgboost import XGBRegressor

import joblib
import math

from scipy.special import logit, expit
from utils import logit_func, expit_func

import optuna
import random
from datetime import datetime

import os

# Set max column width to None.
pd.set_option('display.max_colwidth', None)

# Import dataframe and model.
df = pd.read_csv('df_preds.csv', index_col = 0)    
model = joblib.load('final_model.pkl')

This notebook demonstrates an iterative refinement process: identifying discrepancies between model and sportsbook predictions, investigating their causes, and refining data and model training accordingly. This aligns with best practices for error analysis and improvement.

In [5]:
# Create prediction differential column.
df['pred_diff'] = df['y_pred'] - df['vegas_home_wp']
# df['abs_pred_diff']  = abs(df['y_pred'] - df['vegas_home_wp'])

## Iterative EDA 

Our model's predictions uncovered additional discrepancies in the dataset. We address them here and retrain the model as needed.

In [4]:
# View largest prediction differences.
df.sort_values(by = 'pred_diff', ascending = False).head(10)

,game_id,home_pos,down,ydstogo,yardline_100_home,time_weight,home_score_differential,desc,play_type_nfl,home_spread_line,game_seconds_remaining,home_timeouts_remaining,away_timeouts_remaining,home_win,vegas_home_wp,y_pred,pred_diff
1023155,2020_15_LAC_LV,1,2.0,7.0,24.0,1.0,0.0,"(6:03) 8-M.Mariota pass short right to 28-J.Jacobs to LAC 16 for 8 yards (98-I.Rochell). The Replay Official reviewed the first down ruling, and the play was REVERSED. 8-M.Mariota pass short right to 28-J.Jacobs to LAC 18 for 6 yards (98-I.Rochell).",PASS,-3.5,363.0,2.0,2.0,0,0.493315,0.824467,0.331152
924633,2018_14_BAL_KC,1,1.0,10.0,12.0,1.0,0.0,"(6:25) (Shotgun) 15-P.Mahomes sacked at BAL 25 for -13 yards. FUMBLES, touched at BAL 24, recovered by KC-72-E.Fisher at BAL 25. 72-E.Fisher to BAL 25 for no gain (55-T.Suggs).",SACK,-6.5,385.0,1.0,2.0,1,0.587884,0.918575,0.330691
1023162,2020_15_LAC_LV,1,4.0,5.0,5.0,1.0,0.0,"(3:22) 2-D.Carlson 23 yard field goal is GOOD, Center-47-T.Sieg, Holder-6-A.Cole.",FIELD_GOAL,-3.5,202.0,2.0,1.0,0,0.095640,0.423000,0.327360
1023156,2020_15_LAC_LV,1,3.0,1.0,18.0,1.0,0.0,(5:48) 45-A.Ingold up the middle to LAC 16 for 2 yards (59-N.Vigil).,RUSH,-3.5,348.0,2.0,2.0,0,0.564509,0.868654,0.304144
386631,2007_07_CHI_PHI,1,1.0,10.0,65.0,1.0,-3.0,(:00) 5-D.McNabb pass incomplete short left to 86-R.Brown.,PASS,-5.5,0.0,2.0,0.0,0,0.148071,0.451596,0.303525
870360,2017_11_KC_NYG,1,4.0,5.0,36.0,1.0,0.0,"(2:12) (Shotgun) 10-E.Manning pass deep left to 18-R.Lewis to KC 2 for 34 yards (23-P.Gaines). Penalty on KC-23-P.Gaines, Defensive Pass Interference, declined.",PASS,10.0,132.0,1.0,2.0,1,0.423985,0.719683,0.295698
1081228,2021_20_BUF_KC,1,2.0,6.0,34.0,1.0,0.0,(11:32) (Shotgun) 15-P.Mahomes pass short right to 17-M.Hardman ran ob at BUF 8 for 26 yards (30-D.Jackson).,PASS,-2.5,692.0,2.0,2.0,1,0.467159,0.762239,0.295081
1044895,2021_05_IND_BAL,1,2.0,5.0,5.0,1.0,0.0,"(5:29) (Shotgun) 8-L.Jackson pass short right to 5-M.Brown for 5 yards, TOUCHDOWN.",PASS,-7.5,329.0,2.0,1.0,1,0.630510,0.925032,0.294522
1044890,2021_05_IND_BAL,1,2.0,3.0,26.0,1.0,0.0,(7:42) (Shotgun) 8-L.Jackson up the middle to IND 20 for 6 yards (58-B.Okereke).,RUSH,-7.5,462.0,2.0,2.0,1,0.518335,0.805320,0.286985
1023154,2020_15_LAC_LV,1,1.0,10.0,27.0,1.0,0.0,(6:43) (Shotgun) 28-J.Jacobs left tackle to LAC 24 for 3 yards (56-K.Murray Jr.; 59-N.Vigil).,RUSH,-3.5,403.0,2.0,2.0,0,0.540927,0.822578,0.281652


These investigations highlight cases where sportsbook probabilities may be unreliable. Rather than treating our model as wrong in these instances, we use discrepancies as a diagnostic tool to uncover data quality issues.

In [241]:
# View example.
game = '2020_15_LAC_LV'
df[df['game_id'] == game][['game_id', 'game_seconds_remaining', 'home_score_differential', 'desc', 'play_type_nfl', 'vegas_home_wp', 'y_pred', 'pred_diff']].iloc[-8:-4]

,game_id,game_seconds_remaining,home_score_differential,desc,play_type_nfl,vegas_home_wp,y_pred,pred_diff
1023159,2020_15_LAC_LV,227.0,0.0,(3:47) 28-J.Jacobs right guard to LAC 5 for -2 yards (43-M.Davis).,RUSH,0.766317,0.832162,0.065845
1023161,2020_15_LAC_LV,206.0,0.0,(3:26) 8-M.Mariota pass incomplete short right to 45-A.Ingold (98-I.Rochell).,PASS,0.465657,0.574291,0.108634
1023162,2020_15_LAC_LV,202.0,0.0,"(3:22) 2-D.Carlson 23 yard field goal is GOOD, Center-47-T.Sieg, Holder-6-A.Cole.",FIELD_GOAL,0.095640,0.423000,0.327360
1023164,2020_15_LAC_LV,198.0,3.0,(3:18) 10-J.Herbert pass incomplete short right to 89-D.Parham Jr. [91-K.Vickers].,PASS,0.643072,0.551682,-0.091389


The win probability of .10 for the above field goal is almost certainly invalid, as the plays before and after are .47 and .64, respectively.

Upon evaluating the model's predictions compared to vegas_home_wp, a small number of additional invalid probabilities were located. 

Inspecting the largest 20 variances revealed that each play occurs in overtime. 

In [242]:
# Calculate proportion of overtime plays in dataset.
ot_plays = sum(df['time_weight']==1.0)
print(f'Number of overtime plays: {ot_plays}')
print(f'Proportion of overtime plays: {ot_plays/len(df)}')

Number of overtime plays: 5158
Proportion of overtime plays: 0.0062969324733955214


Because overtime probabilities are inconsistently defined in the dataset and represent a small portion of plays (< 1%), we exclude them to prevent noise from distorting model calibration.

In [208]:
# Re-import cleaned dataset.
df_orig = pd.read_csv('df_filtered.csv', index_col = 0)
# df_orig.head()

In [209]:
# Exclude overtime and confirm.
df_orig = df_orig[df_orig['time_weight'] < 1].copy()
print(f'Number of overtime plays dropped: {len(df)-len(df_orig)}')

Number of overtime plays dropped: 5158


We will again use Optuna to search for the optimal hyperparameters. This retraining ensures that hyperparameters are optimized for the refined dataset, rather than reusing values from earlier runs that included overtime plays.

In [210]:
# Define preprocessing.
features_to_encode = [
    'down'
]

features_to_scale = [
    'yardline_100_home',
    'ydstogo',
    'home_score_differential',
    'home_spread_line'
]

passthrough_features = [
    'home_pos', 
    'time_weight', 
    'home_timeouts_remaining',
    'away_timeouts_remaining']


preprocessing = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(drop='first', sparse_output=False), features_to_encode),
        ('scaler', StandardScaler(), features_to_scale),
        ('noop', FunctionTransformer(validate=False), passthrough_features)  
    ],
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [ ]:
# Set random seed for reproducibility.
np.random.seed(42)
random.seed(42)

X = df.drop('vegas_home_wp', axis = 1)
y = df['vegas_home_wp']

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .2,
                                                    random_state = 42)
# Define objective.
def objective(trial):
    # Define hyperparameter space.
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', .01, .3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 20.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 20.0),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 20),
        'gamma': trial.suggest_int('gamma', 0, 5),
        'random_state': 42
        
    }
   
    # Instantiate model.
    xgb_model_refined = XGBRegressor(random_state = 42)

    model_pipe = Pipeline([
        ('preprocessing', preprocessing),
        ('model', xgb_model_refined)
        ])
    
    # Wrap pipeline in transformed target regressor.
    final_model = TransformedTargetRegressor(
        regressor = model_pipe,
        func = logit_func,
        inverse_func = expit_func
        )

    # Extract parameters from dictionary.
    final_model.set_params(
        **{f'regressor__model__{key}':value for key, value in params.items()})

    # Set scoring to cross validation with negative MSE.
    score = cross_val_score(
        model_pipe,
        X_train,
        y_train,
        cv=3,
        scoring='neg_mean_squared_error',
        n_jobs=1
    )

    return float(score.mean())

# Set save path.
cwd = os.getcwd()
db_path = f"sqlite:///{cwd}/xgb_tuning_04.db"

# Random seed in sampler for reproducibility.
sampler = optuna.samplers.TPESampler(seed=42)

# Instantiate study with sampler.
study = optuna.create_study(
    study_name="xgb_tuning_04",
    direction="maximize",
    storage=db_path,
    load_if_exists=True,
    sampler = sampler
    # skip_if_exists=True
)

# Optimize study.
study.optimize(objective, n_trials = 500)

# View best MSE.
study.best_value

In [ ]:
# View best hyperparameters.
best_params = study.best_params
best_params

In [ ]:
# Explicitly define best hyperparameters.
best_params = {
    'n_estimators': 486,
    'max_depth': 14,
    'learning_rate': 0.07134184802465568,
    'subsample': 0.8309921954982938,
    'colsample_bytree': 0.9371805471715066,
    'colsample_bylevel': 0.9704986984923443,
    'colsample_bynode': 0.8243247029432288,
    'reg_alpha': 0.011694190257938011,
    'reg_lambda': 18.292979039395572,
    'min_child_weight': 20,
    'gamma': 0}

# Print list copy-paste friendly list of hyperparameters.
for k, v in best_params.items():
    print(f'{k} = {v},')

Optuna gave us the hyperparameters 

 - n_estimators = 486,
 - max_depth = 14,
 - learning_rate = 0.07134184802465568,
 - subsample = 0.8309921954982938,
 - colsample_bytree = 0.9371805471715066,
 - colsample_bylevel = 0.9704986984923443,
 - colsample_bynode = 0.8243247029432288,
 - reg_alpha = 0.011694190257938011,
 - reg_lambda = 18.292979039395572,
 - min_child_weight = 20,
 - gamma = 0,

To simulate out-of-sample performance, we use the 2024 season as a holdout set. This prevents temporal leakage and evaluates how well the model generalizes to unseen seasons.

In [243]:
# Training set 2023 and older.
df_train = df_orig[~df_orig['game_id'].str.contains("2024")].copy()

# Test set 2024.
df_test = df_orig[df_orig['game_id'].str.contains("2024")].copy()


X_train = df_train.drop('vegas_home_wp', axis = 1)
X_test = df_test.drop('vegas_home_wp', axis = 1)

y_train = df_train['vegas_home_wp']
y_test = df_test['vegas_home_wp']


# Instantiate model with optimal hyperparameters.
final_model = XGBRegressor(
    n_estimators = 486,
    max_depth = 14,
    learning_rate = 0.07134184802465568,
    subsample = 0.8309921954982938,
    colsample_bytree = 0.9371805471715066,
    colsample_bylevel = 0.9704986984923443,
    colsample_bynode = 0.8243247029432288,
    reg_alpha = 0.011694190257938011,
    reg_lambda = 18.292979039395572,
    min_child_weight = 20,
    gamma = 0,
)

# Wrap model in pipeline.
model_pipe = Pipeline([
    ('preprocessing', preprocessing),
    ('model', final_model)
])

# Feed pipeline into transformed target regressor.
ui_model = TransformedTargetRegressor(
    regressor = model_pipe,
    func = logit_func,
    inverse_func = expit_func,
    check_inverse = False
)

# Fit model
ui_model.fit(X_train, y_train)

# Predict.
y_pred = ui_model.predict(X_test)

# View Results.
print(f'Updated R²: {r2_score(y_test, y_pred)}')
print(f'Updated MSE: {mean_squared_error(y_test, y_pred)}')

Updated R²: 0.9966759519369123
Updated MSE: 0.0003253792843940762


We repeat the feature engineering from above and export our train and test dataframes, along with the pickled model.

In [247]:
# Predict on training data (2003-2023).
y_pred_train = ui_model.predict(X_train)

# Create training dataframe with predictions.
df_03_23 = pd.concat([df_train, pd.Series(y_pred_train, index = 
                                         df_train.index, 
                                         name = 'y_pred')], axis = 1)


# Create test dataframe with predictions.
df_24 = pd.concat([df_test, pd.Series(y_pred, index = 
                                         df_test.index, 
                                         name = 'y_pred')], axis = 1)

# Create prediction differential columns in each dataframe.
df_03_23['pred_diff'] = df_03_23['y_pred'] - df_03_23['vegas_home_wp']
df_24['pred_diff'] = df_24['y_pred'] - df_24['vegas_home_wp']

# Export train and test dataframes.
df_03_23.to_csv('df_03_23.csv')
df_24.to_csv('df_24.csv')

# Export model.
joblib.dump(ui_model, 'ui_model.pkl', compress=3)

## Iterative Refinement Summary

 - Model prediction discrepancies flagged potential data issues (e.g., invalid win probabilities on certain plays).
 - Invalid win probabilities were isolated to overtime.
 - Overtime plays were removed.
 - Hyperparameters were re-optimized with Optuna on the refined dataset.
 - A temporal split (2003-2023 training, 2024 testing) was adopted to ensure realistic generalization.

These refinements demonstrate how iterative error analysis strengthens dataset reliability.